# 03 — Machine Learning baseline on Helmholtz data

Goal: train a small CNN/FNO to map inputs → complex fields and report robust metrics.

We’ll:
1) build a synthetic dataset by solving PDEs,
2) train a tiny model (LocalCNN by default),
3) evaluate rel-L2 (mean/median/p90) + magnitude/phase RMSE,
4) visualize predictions vs targets,
5) (optional) save weights and dataset.


In [1]:
import os, sys, importlib, inspect

PROJECT_ROOT = r"C:\Users\31624\Documents\MIT\Programming\FreqTransfer"
os.chdir(PROJECT_ROOT)

# put the project root FIRST so it wins any name conflicts
if PROJECT_ROOT in sys.path:
    sys.path.remove(PROJECT_ROOT)
sys.path.insert(0, PROJECT_ROOT)

# hard reload src and show where it came from
import src
importlib.reload(src)
print("src file:", src.__file__)
print("has __getattr__ (lazy ML)?", hasattr(src, "__getattr__"))


src file: C:\Users\31624\Documents\MIT\Programming\FreqTransfer\src\__init__.py
has __getattr__ (lazy ML)? True


In [6]:
import src.ml as ml
importlib.reload(ml)
print("ml file:", ml.__file__)
print("exports include build_direct_map?", hasattr(ml, "build_direct_map"))


AttributeError: 'NoneType' object has no attribute 'Module'

In [2]:
# python
import importlib.util, sys, traceback, importlib
importlib.invalidate_caches()

print("find_spec('src') ->", importlib.util.find_spec("src"))
spec = importlib.util.find_spec("src")
if spec:
    print(" spec.origin:", spec.origin)
    print(" spec.loader:", spec.loader)

try:
    import src as _src
    print("Imported src from:", getattr(_src, '__file__', None))
except Exception:
    traceback.print_exc()

find_spec('src') -> ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001629065BE00>, origin='C:\\Users\\31624\\Documents\\MIT\\Programming\\FreqTransfer\\src\\__init__.py', submodule_search_locations=['C:\\Users\\31624\\Documents\\MIT\\Programming\\FreqTransfer\\src'])
 spec.origin: C:\Users\31624\Documents\MIT\Programming\FreqTransfer\src\__init__.py
 spec.loader: <_frozen_importlib_external.SourceFileLoader object at 0x000001629065BE00>
Imported src from: C:\Users\31624\Documents\MIT\Programming\FreqTransfer\src\__init__.py


In [3]:
# python
import sys, types
print("sys.modules['types']:", sys.modules.get('types'))
print("types module file:", getattr(types, '__file__', None))
# any local file named types.py / importlib.py / typing.py in project?
import pathlib
root = pathlib.Path.cwd()
candidates = list(root.rglob("types.py")) + list(root.rglob("importlib.py")) + list(root.rglob("typing.py"))
print("potential shadowing files (relative):", [p.relative_to(root) for p in candidates])

sys.modules['types']: <module 'types' from 'C:\\Users\\31624\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\types.py'>
types module file: C:\Users\31624\AppData\Local\Programs\Python\Python312\Lib\types.py
potential shadowing files (relative): [WindowsPath('.venv/Lib/site-packages/contourpy/types.py'), WindowsPath('.venv/Lib/site-packages/fontTools/designspaceLib/types.py'), WindowsPath('.venv/Lib/site-packages/matplotlib/typing.py'), WindowsPath('.venv/Lib/site-packages/jedi/inference/gradual/typing.py')]


In [3]:
from src import GridSpec
from src import build_freq_transfer, LocalCNN, SimpleFNO
from src import train_model, eval_relative_metrics
print("OK")


ImportError: cannot import name 'build_freq_transfer' from 'src' (C:\Users\31624\Documents\MIT\Programming\FreqTransfer\src\__init__.py)

In [5]:
from src import GridSpec
from src import build_direct_map, build_freq_transfer, LocalCNN, SimpleFNO
from src import train_model, eval_relative_metrics

# Torch check
try:
    import torch
    print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
except Exception as e:
    print("PyTorch not installed. Install with: pip install torch torchvision")
    raise


ImportError: cannot import name 'build_direct_map' from 'src' (C:\Users\31624\Documents\MIT\Programming\FreqTransfer\src\__init__.py)

## Experiment knobs

- `task`: choose **"direct"** (RHS→u) or **"transfer"** (u(ω)→u(ω′)).
- Dataset size is kept small for speed; bump when happy.
- Use **LocalCNN** for quick runs; **SimpleFNO** is slower but stronger.


In [6]:
# --- task selection ---
TASK = "direct"        # "direct" or "transfer"

# --- grid & data ---
grid = GridSpec(2, (48, 48), (1.0, 1.0))
N_SAMPLES = 128        # keep small for quick runs; try 512+ later
K_RANGE = (20.0, 40.0) # for "direct"
OMEGA, OMEGA_P = 25.0, 35.0  # for "transfer"

# --- model ---
MODEL_NAME = "local"   # "local" or "fno"
WIDTH = 48             # channels in the hidden layers

# --- training ---
EPOCHS = 5
BATCH_SIZE = 8
LR = 1e-3
SEED = 0

# --- saving ---
SAVE_DIR = "data/results/03_ml"
os.makedirs(SAVE_DIR, exist_ok=True)


## Build dataset

- **Direct map:** synthesize pairs (RHS[, k]) → solution by assembling and solving.
- **Transfer map:** solve at ω and ω′ with the same RHS, learning u(ω) → u(ω′).


In [ ]:
import numpy as np
np.random.seed(SEED)

if TASK == "direct":
    ds = build_direct_map(
        n=N_SAMPLES,
        grid=grid,
        k_range=K_RANGE,
        include_k_channel=True,
        seed=SEED,
        gmres_tol=1e-6,
    )
elif TASK == "transfer":
    ds = build_freq_transfer(
        n=N_SAMPLES,
        grid=grid,
        omega=OMEGA,
        omega_p=OMEGA_P,
        seed=SEED,
        gmres_tol=1e-6,
    )
else:
    raise ValueError("TASK must be 'direct' or 'transfer'")

len(ds), ds[0][0].shape, ds[0][1].shape  # (N, (C_in,H,W), (2,H,W))


## Peek at one sample

Check the tensor shapes:
- **Inputs:** channels are `[Re(rhs), Im(rhs), k]` for direct, or `[Re(u_ω), Im(u_ω)]` for transfer.
- **Targets:** `[Re(u), Im(u)]`.


In [ ]:
in_ch = ds[0][0].shape[0]
if MODEL_NAME == "local":
    model = LocalCNN(in_ch=in_ch, width=WIDTH)
elif MODEL_NAME == "fno":
    model = SimpleFNO(in_ch=in_ch, width=WIDTH, modes=(12,12), layers=4)
else:
    raise ValueError("MODEL_NAME must be 'local' or 'fno'")

model


## Train

We use MSE loss on the two output channels `[Re, Im]`.  
A small train/val split monitors overfitting. Keep epochs small until the loop is stable.


In [7]:
model, history = train_model(
    model,
    ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    lr=LR,
    val_split=0.2,
    verbose=True,
)
history


NameError: name 'train_model' is not defined

## Evaluate metrics

We report:
- **rel L2** mean / median / p90 across samples  
- **magnitude RMSE** and **phase RMSE** (phase difference wrapped to \([-π,π]\))


In [ ]:
metrics = eval_relative_metrics(model, ds, batch_size=16)
metrics


## Visualize predictions vs targets

Pick a few samples and compare magnitude & phase qualitatively.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

def show_example(i=0):
    x, y = ds[i]            # tensors
    with torch.no_grad():
        yhat = model(x.unsqueeze(0)).squeeze(0).cpu().numpy()  # (2,H,W)

    yt = y.cpu().numpy()
    def to_complex(twoch): return twoch[0] + 1j*twoch[1]

    u_pred = to_complex(yhat)
    u_true = to_complex(yt)

    fig, axes = plt.subplots(2, 2, figsize=(8,6))
    im0 = axes[0,0].imshow(np.abs(u_true), origin="lower"); axes[0,0].set_title("|u| true"); plt.colorbar(im0, ax=axes[0,0])
    im1 = axes[0,1].imshow(np.abs(u_pred), origin="lower"); axes[0,1].set_title("|u| pred"); plt.colorbar(im1, ax=axes[0,1])
    im2 = axes[1,0].imshow(np.angle(u_true), origin="lower"); axes[1,0].set_title("phase true"); plt.colorbar(im2, ax=axes[1,0])
    # wrap phase diff to [-pi,pi] for a more informative view
    dphi = np.angle(u_pred) - np.angle(u_true)
    dphi = np.angle(np.exp(1j*dphi))
    im3 = axes[1,1].imshow(dphi, origin="lower"); axes[1,1].set_title("phase diff"); plt.colorbar(im3, ax=axes[1,1])
    for ax in axes.ravel(): ax.set_xticks([]); ax.set_yticks([])
    plt.tight_layout()

show_example(i=0)


## Save model & config (optional)

Save trained weights and experiment settings for reproducibility.


In [ ]:
torch.save(model.state_dict(), os.path.join(SAVE_DIR, f"{MODEL_NAME}_{TASK}_weights.pt"))
with open(os.path.join(SAVE_DIR, "params.txt"), "w") as f:
    f.write(
        f"TASK={TASK}\nGRID={grid.shape}\nN_SAMPLES={N_SAMPLES}\n"
        f"K_RANGE={K_RANGE}\nOMEGA={OMEGA}, OMEGA_P={OMEGA_P}\n"
        f"MODEL={MODEL_NAME}, WIDTH={WIDTH}\nEPOCHS={EPOCHS}, BATCH={BATCH_SIZE}, LR={LR}\n"
    )
print("Saved to:", SAVE_DIR)


## Troubleshooting

- **PyTorch missing:** `pip install torch torchvision` (match your Python and CUDA).
- **Slow dataset build:** reduce `N_SAMPLES`, use smaller grids (e.g., 40×40), or increase GMRES tolerance during data synthesis to `1e-5`/`1e-4` temporarily.
- **OOM during training:** lower `WIDTH`, use `BATCH_SIZE=4`, or switch to `LocalCNN`.
- **Metrics look too good/bad:** visualize a few examples (`show_example(i=...)`) to understand failure modes (phase wrapping, boundary effects, etc.).


In [ ]:
import importlib.util, sys
spec = importlib.util.find_spec("src")
print(spec.origin if spec else "src not found")
